In [ ]:
import scipy.signal as signal
import numpy as np
from numpy.random import randn
import matplotlib.pyplot as plt
import timeit

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = 16,6

In [ ]:
def time_func(func, N, M):
    x = randn(N)
    h = signal.firwin(numtaps=10*M, cutoff=1./M)
    if func is signal.decimate:
        filt = signal.dlti(h, 1)
        t = %timeit -q -o -r 2 func(x=x, q=M, ftype=filt)
    elif func is signal.resample_poly:
        t = %timeit -q -o -r 2 func(x=x, up=1, down=M, window=h)
    else:
        t = %timeit -q -o -r 2 func(x, h, M)
    return t.average

In [ ]:
def benchmark_func(func, N_vec, M_vec):
    times = np.zeros((len(M_vec), len(N_vec)))
    for i in range(len(M_vec)):
        for j in range(len(N_vec)):
            M = M_vec[i]
            N = N_vec[j]
            if 10*M > N:
                times[i,j] = np.nan
            else:
                time = time_func(func, N, M)
                times[i,j] = time * 1e3
#                 print("({},{}) -- {:0.2f} ms".format(N, M, time * 1e3))
    return times

In [ ]:
def filter_then_decimate(x, h, M):
    # Inputs:  1-D numpy array, x, an input signal at rate fs
    #          1-D numpy array, h, an anti-aliasing lowpass filter
    #          int, M, the decimation factor
    # Outputs: 1-D numpy array, y, the output signal at rate fs/M
    
    v = ??? # filter x with h using direct convolution
    y = v[M-1:len(x):M] # decimate by M (and match input length)
    
    return y

In [ ]:
def polyphase_downsample(x, h, M):
    # Inputs:  1-D numpy array, x, an input signal at rate fs
    #          1-D numpy array, h, an anti-aliasing lowpass filter
    #          int, M, the decimation factor
    # Outputs: 1-D numpy array, y, the output signal at rate fs/M
    
    y = np.zeros(len(x)//M)
    for k in range(M):
        xk = ??? # kth subsampled signal
        ek = ??? # kth subsampled filter (need to flip because of convolution definition)
        yk = ??? # filter xk with ek using direct convolution
        y = ??? # add output contribution to final output
    
    return y

In [ ]:
# check output for correctness
N = 2 ** 10
M = 2 ** 5
x = randn(N)
h = signal.firwin(numtaps=10*M, cutoff=1./M)
y = polyphase_downsample(x, h, M)
y0 = filter_then_decimate(x, h, M)
rmse = np.sqrt( (abs(y - y0) ** 2).mean() )
print("RMSE = ", rmse)

In [ ]:
# benchmark functions (SLOW! May take a few minutes on the Pi.)
N_vec = 2 ** np.array([6,8,10,12,14,16])
M_vec = 2 ** np.array([2,6,10])
times_ftd = benchmark_func(filter_then_decimate, N_vec, M_vec)
times_poly = benchmark_func(polyphase_downsample, N_vec, M_vec)
times_ftd_s = benchmark_func(signal.decimate, N_vec, M_vec)
times_poly_s = benchmark_func(signal.resample_poly, N_vec, M_vec)

In [ ]:
# plot benchmark data
for i in range(len(M_vec)):
    fig, ax = plt.subplots()
    plt.loglog(N_vec, times_ftd[i], '.-')
    plt.loglog(N_vec, times_poly[i], '.-')
    plt.loglog(N_vec, times_ftd_s[i], '.-')
    plt.loglog(N_vec, times_poly_s[i], '.-')
    plt.legend(["Filter then decimate (mine)", "Polyphase downsample (mine)", "Filter then decimate (Scipy's)", "Polyphase downsample (Scipy's)"])
    plt.title("Decimation factor = {}".format(M_vec[i]))
    plt.xlabel("Input signal size")
    plt.ylabel("Time [ms]")
    plt.show()

#### Some questions:
1. For very short inputs and low decimation factors, which of your implementations is faster?
2. For high decimation factors, which of your implementations is faster?
3. Scipy's filter-then-decimate is almost identical to Scipy's polyphase downsampling in speed. Why might this be? In what setting is polyphase downsampling most useful?
4. How do your implementations compare to Scipy's built-in functions?

#### Your answers here:
